In [ ]:
import torch
import torch.nn as nn
import numpy as np
import torch
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm


In [ ]:
# ------------------------ Definizione della rete neurale ------------------------
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        # Rete fully connected con 2 hidden layers e funzione di attivazione Tanh
        self.net = nn.Sequential(
            nn.Linear(4, 64),    # input: x0, x1, mu0, mu1
            nn.Tanh(),
            nn.Linear(64, 64),
            nn.Tanh(),
            nn.Linear(64, 1)     # output: u(x, mu)
        )

    def forward(self, xmu):
        # Forward pass: calcola u dato input (x0,x1,mu0,mu1)
        return self.net(xmu)

In [ ]:
# ------------------------ Calcolo del residuo PDE ------------------------
def pde_residual(xmu, net):
    """
    Calcola il residuo della PDE in ogni punto (interno al dominio)
    -xmu: tensor Nx4 con (x0, x1, mu0, mu1)
    -net: la rete neurale che approssima u
    """
    xmu.requires_grad_(True)      # necessario per calcolare derivate rispetto a xmu
    u = net(xmu)                 # output della rete: u(x,mu)

    # Calcolo gradiente di u rispetto a input (x0, x1) per le derivate prime
    grads = torch.autograd.grad(u.sum(), xmu, create_graph=True)[0]
    u_x0 = grads[:, 0:1]          # ∂u/∂x0
    u_x1 = grads[:, 1:2]          # ∂u/∂x1

    # Calcolo derivate seconde per laplaciano
    u_x0x0 = torch.autograd.grad(u_x0.sum(), xmu, create_graph=True)[0][:, 0:1]  # ∂²u/∂x0²
    u_x1x1 = torch.autograd.grad(u_x1.sum(), xmu, create_graph=True)[0][:, 1:2]  # ∂²u/∂x1²
    laplacian_u = u_x0x0 + u_x1x1  # ∆u

    # Estrazione parametri mu0 e mu1
    mu0 = xmu[:, 2:3]
    mu1 = xmu[:, 3:4]

    # Coordinate spaziali
    x0 = xmu[:, 0:1]
    x1 = xmu[:, 1:2]

    # Termine sorgente g(x;mu)
    g = 100 * torch.sin(2 * np.pi * x0) * torch.cos(2 * np.pi * x1)

    # Termine non lineare con mu0, mu1
    nonlinear = (mu0 / mu1) * (torch.exp(mu1 * u) - 1)

    # Residuo PDE: -∆u + nonlinear - g = 0
    residual = -laplacian_u + nonlinear - g
    return residual

In [ ]:
# ------------------------ Generazione punti dominio e bordo ------------------------
def generate_domain_points(N_interior, N_boundary, mu0_range, mu1_range):
    """
    Genera punti interni e di bordo del dominio con campionamento uniforme
    anche dei parametri mu0, mu1 negli intervalli specificati
    """
    mu0_min, mu0_max = mu0_range
    mu1_min, mu1_max = mu1_range

    # --- Punti interni ---
    x0 = torch.rand(N_interior, 1)  # coordinate x0 in (0,1)
    x1 = torch.rand(N_interior, 1)  # coordinate x1 in (0,1)
    mu0 = mu0_min + (mu0_max - mu0_min) * torch.rand(N_interior, 1)  # parametri mu0 casuali nell’intervallo
    mu1 = mu1_min + (mu1_max - mu1_min) * torch.rand(N_interior, 1)  # parametri mu1 casuali nell’intervallo
    xmu_interior = torch.cat([x0, x1, mu0, mu1], dim=1)

    # --- Punti sul bordo ---
    xb = []
    for side in range(4):
        s = torch.rand(N_boundary, 1)          # coordinata variabile sul lato
        zeros = torch.zeros_like(s)             # vettore di zeri
        ones = torch.ones_like(s)               # vettore di uni

        if side == 0:
            # lato inferiore y=0
            x0b, x1b = s, zeros
        elif side == 1:
            # lato superiore y=1
            x0b, x1b = s, ones
        elif side == 2:
            # lato sinistro x=0
            x0b, x1b = zeros, s
        else:
            # lato destro x=1
            x0b, x1b = ones, s

        mu0b = mu0_min + (mu0_max - mu0_min) * torch.rand(N_boundary, 1)  # mu0 bordo
        mu1b = mu1_min + (mu1_max - mu1_min) * torch.rand(N_boundary, 1)  # mu1 bordo

        xb.append(torch.cat([x0b, x1b, mu0b, mu1b], dim=1))

    xmu_boundary = torch.cat(xb, dim=0)
    n_boundary_points = len(xmu_boundary)



    return xmu_interior, xmu_boundary

In [ ]:
# ------------------------ Funzione di training ------------------------
def train_pinn(net, epochs=5000, N_interior=1000, N_boundary=200, lr=1e-3,
               mu0_range=(0.1, 1.0), mu1_range=(0.1, 1.0)):
    """
    Addestra la rete PINN minimizzando residuo PDE e rispettando condizioni di Dirichlet
    """
    optimizer = torch.optim.Adam(net.parameters(), lr=lr)

    for epoch in range(1, epochs + 1):

        optimizer.zero_grad()
        net.train()
        # Genera punti (interni e bordo) con mu0, mu1 random su intervalli
        xmu_int, xmu_bnd = generate_domain_points(N_interior, N_boundary, mu0_range, mu1_range)

        # Calcola residuo PDE sui punti interni
        res_int = pde_residual(xmu_int, net)
        loss_pde = torch.mean(res_int ** 2)

        # Valuta la rete sui punti di bordo (condizione Dirichlet u=0)
        u_bnd = net(xmu_bnd)
        # qua residuo è u_bnd - 0
        loss_bc = torch.mean(u_bnd ** 2)

        # Loss totale: somma tra PDE residual e condizione al contorno (dovrei aggiungere un lambda)
        loss =  loss_pde +  loss_bc

        loss.backward()
        optimizer.step()

        if epoch % 500 == 0 or epoch == 1:
            print(f"Epoch {epoch}: Loss = {loss.item():.6f} (PDE = {loss_pde.item():.6f}, BC = {loss_bc.item():.6f})")

In [ ]:
net = Net()
train_pinn(net,
           epochs=10000,
           N_interior=1000,
           N_boundary=200,
           lr=1e-3,
           mu0_range=(0.1, 1.0),
           mu1_range=(0.1, 1.0))


In [ ]:
# Fissa parametri mu0 e mu1
mu0_val = 0.5
mu1_val = 0.5

# Griglia su (x0, x1)
x0 = np.arange(0, 1.01, 0.02)
x1 = np.arange(0, 1.01, 0.02)
X0, X1 = np.meshgrid(x0, x1)

# Prepara input per la rete: (x0, x1, mu0, mu1)
x0_flat = X0.flatten()[:, None]
x1_flat = X1.flatten()[:, None]
mu0_flat = mu0_val * np.ones_like(x0_flat)
mu1_flat = mu1_val * np.ones_like(x0_flat)

inputs = np.hstack([x0_flat, x1_flat, mu0_flat, mu1_flat])
inputs_torch = torch.from_numpy(inputs).float()

# Predizione rete (assumendo 'net' già addestrata e in eval mode)
net.eval()
with torch.no_grad():
    u_pred = net(inputs_torch).cpu().numpy()

# Ricostruisci la matrice della soluzione
U = u_pred.reshape(X0.shape)

# Plot 3D
fig = plt.figure(figsize=(8,6))
ax = fig.add_subplot(111, projection='3d')

surf = ax.plot_surface(X0, X1, U, cmap=cm.coolwarm, linewidth=0, antialiased=False)
ax.set_xlabel('$x_0$')
ax.set_ylabel('$x_1$')
ax.set_zlabel('$u(x_0,x_1)$')
ax.set_title(f'Soluzione PINN con $\\mu_0={mu0_val}$, $\\mu_1={mu1_val}$')

fig.colorbar(surf, shrink=0.5, aspect=5)
plt.show()